In [1]:
from itertools import count

import pandas as pd
from pandas import Index

pd.__version__

'2.2.2'

Datei einlesen

In [3]:
df = pd.read_excel('ZA8732_v1-0-0.xlsx')
question_regex_filter = r'[f]\d+_\d+'
questions = df.filter(regex=question_regex_filter, axis="columns")
personal_data_regex_filter=r'[s]\d'
personal_data = df.filter(regex=personal_data_regex_filter, axis="columns")

col_bundesland_index = 's3'
col_bundesland = personal_data[col_bundesland_index]

df

FileNotFoundError: [Errno 2] No such file or directory: 'data/ZA8732_v1-0-0.xlsx'

## Define order of answers (all answers in one)
Simply add answers to this array

In [ ]:
answers_ordered = [
    "Weiß nicht/ Keine Angabe",
    "gar nicht",
    "weniger stark",
    "stark",
    "sehr stark",
    "Antwort1",
    "Antwort2",
    "Antwort3",
    "Antwort4",
    "Antwort5",
];

In [ ]:
from ipywidgets import widgets
question_selection = widgets.Dropdown(
    options=questions.columns,
    value=questions.columns[0],
    description='Question:',
    disabled=False
)
display(question_selection)

names_germany_west = [
    'Hamburg',
    "Schleswig Holstein",
    "Nordrhein Westfalen",
    "Bayern",
    "Baden Württemberg",
    "Niedersachsen",
    "Hessen",
    "Rheinland Pfalz",
    "Bremen",
    "Saarland"]
names_germany_east = ["Berlin", "Sachsen", "Thüringen", "Bandenburg", "Mecklenburg Vorpommern", "Sachsen Anhalt"]
names_germany_west.sort()
names_germany_east.sort()

bundeslander_values = col_bundesland.value_counts()
question_country_selector = widgets.SelectMultiple(
    options=sorted(bundeslander_values.keys()),
    value=[bundeslander_values.keys()[0]],
    description='Bundesland:',
    disabled=False
)
bundeslander_presets = widgets.ToggleButtons(
    options=['West', 'East', 'Free Selection'],
    description='Preset:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast']
)
def on_preset_change(name):
    question_country_selector.value = names_germany_west if name.value == 'West' else names_germany_east
    print(name)
bundeslander_presets.observe(on_preset_change, names='value')
display(bundeslander_presets)
display(question_country_selector)



# Prepare dataframe for specific answer

In [ ]:
country_question_df = df[[col_bundesland_index, question_selection.value]]
df_west = country_question_df.loc[df[col_bundesland_index].isin(names_germany_west)]
df_east = country_question_df.loc[df[col_bundesland_index].isin(names_germany_east)]

value_counts_series = {'All': country_question_df[question_selection.value].value_counts(), 'West': df_west[question_selection.value].value_counts(), 'East': df_east[question_selection.value].value_counts()}
for bun in [*names_germany_west, *names_germany_east]:
    value_counts_series[bun] = country_question_df.loc[df[col_bundesland_index] == bun][question_selection.value].value_counts()

answers_df = pd.DataFrame(value_counts_series)
sorted_labels = [label for label in answers_ordered if label in answers_df.index]
sorted_labels = sorted_labels + [label for label in answers_df.index if label not in sorted_labels]
answers_df = answers_df.reindex(sorted_labels)
answers_df = answers_df.fillna(0).astype(int) # Sort based on ordered list

display(sorted_labels)
display(answers_df.loc[:,question_country_selector.value])
display(answers_df)
